In [6]:
import newspaper
from PyCrowlingo import Client
from PyCrowlingo.Errors import ModelNotFound
from tqdm import tqdm


In [7]:
client = Client("<TOKEN>")

model_id = "newspaper_search_engine"
batch_size = 5


In [8]:
try:
    client.model.delete(model_id)
except ModelNotFound as e:
    print(e)
client.model.create(model_id, "search_engine")


Create(model_id='W4bGKufu:')

In [9]:
for url in ["https://lemonde.fr", "https://bbc.com"]:
    np = newspaper.build(url, memoize_articles=False)
    docs = []
    print(f"Analyze {url}")
    for i, article in tqdm(enumerate(np.articles[:250])):
        try:
            article.download()
            article.parse()
        except:
            pass
        # print(f"Index {url} - {i}: {article.title}")
        docs.append({"text": f"{article.title}\n {article.text}"})
        if len(docs) >= batch_size:
            client.search_engine.create_documents(model_id, docs, light=True)
            docs = []
    client.search_engine.create_documents(model_id, docs)

0it [00:00, ?it/s]

Analyze https://lemonde.fr


250it [01:27,  2.86it/s]
0it [00:00, ?it/s]

Analyze https://bbc.com


250it [01:38,  2.54it/s]


In [10]:
client.search_engine.wait_indexing(model_id)

100%|██████████| 5/5 [00:03<00:00,  1.60it/s]


{'fr': 232,
 'en': 239,
 'ky': 4,
 'my': 3,
 'rw': 4,
 'so': 1,
 'ps': 3,
 'si': 1,
 'sw': 3}

In [13]:
client.search_engine.search(model_id, "attack", limit=2, languages=["en", "fr"])

Search(keywords={'en': [{'weight': 0.9999997615814351, 'text': 'attack'}, {'weight': 0.749232386551185, 'text': 'combat'}], 'fr': [{'weight': 0.8971693225711341, 'text': 'attaque'}, {'weight': 0.8526222905974379, 'text': 'attaques'}]})